In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import pandas as pd

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

# 페이지 반복
cafe = []
for i in range(1, 2):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=recentdate&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)

    blog_links = []

    # 한페이지에 7개 블로그 글
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(
                By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner"
            )
            blog_links.append(elem.get_attribute("href"))
        except:
            continue

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(
                f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}"
            )
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR, "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                # 블로그 내용 크롤링
                blog = driver.find_element(
                    By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc"
                ).text

                cafe.append(
                    {
                        "logNo": logNo,
                        "카페이름": map_data["name"],
                        "카페주소": map_data["address"],
                        "위도": map_data["latitude"],
                        "경도": map_data["longitude"],
                        "블로그내용": blog,
                    }
                )

            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue

지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음


In [3]:
cafe

[{'logNo': '223944317831',
  '카페이름': '미쁘동 연남본점',
  '카페주소': '서울특별시 마포구 동교로38길 33-21 2층',
  '위도': '37.5621119',
  '경도': '126.9262048',
  '블로그내용': '대부분 전체공개\n2025 June\n오이 ・ 31분 전\nURL 복사 이웃추가\n본문 기타 기능\n나미가 일본다녀와서\n회덮밥먹구싶다고 데려간 곳\n이렇게 조사진 회덮밥이 먹고싶대서 갔는데\n미 미.....반쯤먹고 육수에 말아주는것도 너무너무취향이었다...\n50m\n© NAVER Corp.\n미쁘동 연남본점\n서울특별시 마포구 동교로38길 33-21 2층\n웨이팅있을까봐 걱정했는데\n바로앞에 2점이 있어서 유연하게 안내하더라\n좋았음!\n나미의 선물..♡\n댈구요청한 피크민친구들과\n젤리>.<\n글고..!!! 나의관념적담배 민티아 드라이하드!!!!!!!!♡♡♡♡ 이거요새 하루에 오천알씩먹고잇음\n폼폼푸린빙수\n빙수조아\n앞에 핑구도 나미가 뽑아줌\n내 최애는 핑아인데\n물개만아니면돼~ 했는데 1트가 물개가나왔댄다\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n그래도 나미가 다뽑아줘서 넘조아.ㅎㅎ\n하 명주가 이런머리하고잇엇던게\n너무너무이뻐서 손민수하고싶어서\n퇴근하고 미용실갓더니 예약안했음 안된대ㅡㅡㅅㅂ먼\n짱나서 집와서 걍 냅다 자름\n히히\n얼추비슷하지않나용ㅋ\n전 넘맘에들어용~~\n투표했다\n드디어..ㅡ\n미친놈들진짜 한번에잘하자고 개새끼들아\n조별과제존나안돼미친놈들\n제발\n사람다운사람을 뽑으라고진짜 뭐가문제지진짜\n이재명씨 앞으로 잘부탁드려요.\n투표 끝나고 가족외식으로 송도갈비에 갔음!\n송도갈비먹으면 바로 옆 카페인 바다쏭의 할인권을 줌!\n저번엔 바다쏭을 갔었는데 넘좋앗어가지고~~\n밥먹구 카페까지 갔다가 옴~ㅎㅎ\n피크민 6월 새로바뀐 메인이 넘이쁨\n그리고..세상어디서도 볼 수 없는 나의 피크민 꽃심기 조합\n전직장동료 - 버섯 자만추한 피 - 뜨친의 친구 - 내트친 - 나\n가족여행에 갔다왔습니다\n너

In [ ]:
df_cafe = pd.DataFrame(cafe)
df_cafe

In [ ]:
# sql

In [4]:
import pymysql

In [5]:
envs = {}
with open(".env", "r") as f:
    for line in f.readlines():
        key, value = line.rstrip().split("=")
        envs[key] = value

conn = pymysql.connect(
    host=envs["DB_HOST"],
    port=int(envs["DB_PORT"]),
    user=envs["DB_USER"],
    password=envs["DB_PASSWORD"],
    database=envs["DB_DATABASE"],
    charset="utf8mb4",
)

In [7]:
with conn.cursor() as cur:
    sql = "INSERT INTO tb_cafe (logNo,cafename,cafeaddress,latitude,longitude,blogtext) VALUES (%s,%s,%s,%s,%s,%s)"

    values = [
        (
            data["logNo"],
            data["카페이름"],
            data["카페주소"],
            float(data["위도"]),
            float(data["경도"]),
            data["블로그내용"],
        )
        for data in cafe
    ]

    cur.executemany(sql, values)
conn.commit()

In [16]:
import pymysql.cursors # 필드값 까지 가져오게 됨 

with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur: 
    result = cur.execute('SELECT id, logNo, cafename FROM tb_cafe ORDER BY logNo ASC')
    datas = cur.fetchall()
datas

[{'id': 3, 'logNo': '223944255718', 'cafename': '맨리 비치'},
 {'id': 2, 'logNo': '223944257143', 'cafename': '누뗀'},
 {'id': 1, 'logNo': '223944317831', 'cafename': '미쁘동 연남본점'}]